In [1]:
from ultralytics import YOLO
model = YOLO('best.pt')

In [30]:
import os
from PIL import Image

def process_image(image_path):
    try:
        img = Image.open(image_path)
        results = model(img, conf=0.5)
        prediction = sum(len(result.boxes.xyxy) for result in results)
        img.close()
        return prediction
    except Exception as e:
        print(f"Error processing image '{image_path}': {e}")
        return None

def process_images(directory_path):
    predictions = []

    # Check if the directory exists
    if not os.path.exists(directory_path):
        print(f"Directory '{directory_path}' does not exist.")
        return predictions

    # Get all files in the directory
    files = os.listdir(directory_path)

    # Filter only image files (you can extend this list based on the image formats you want to include)
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp']
    image_files = [file for file in files if any(file.lower().endswith(ext) for ext in image_extensions)]

    # Process each image individually
    for image_file in image_files:
        img_id = image_file.split('.')[0]
        image_path = os.path.join(directory_path, image_file)
        prediction = process_image(image_path)
        if prediction is not None:
            predictions.append(prediction)

    return predictions

# Example usage
directory_path = 'uav/test/images/'
predictions = process_images(directory_path)


0: 704x1024 262 Maizes, 274.7ms
Speed: 46.0ms preprocess, 274.7ms inference, 2.0ms postprocess per image at shape (1, 3, 704, 1024)

0: 704x1024 141 Maizes, 49.5ms
Speed: 5.1ms preprocess, 49.5ms inference, 1.0ms postprocess per image at shape (1, 3, 704, 1024)

0: 704x1024 185 Maizes, 46.1ms
Speed: 5.0ms preprocess, 46.1ms inference, 3.0ms postprocess per image at shape (1, 3, 704, 1024)

0: 704x1024 124 Maizes, 45.5ms
Speed: 5.0ms preprocess, 45.5ms inference, 2.0ms postprocess per image at shape (1, 3, 704, 1024)

0: 704x1024 151 Maizes, 45.8ms
Speed: 5.0ms preprocess, 45.8ms inference, 2.0ms postprocess per image at shape (1, 3, 704, 1024)

0: 704x1024 110 Maizes, 46.0ms
Speed: 4.0ms preprocess, 46.0ms inference, 2.0ms postprocess per image at shape (1, 3, 704, 1024)

0: 704x1024 282 Maizes, 47.0ms
Speed: 4.0ms preprocess, 47.0ms inference, 2.0ms postprocess per image at shape (1, 3, 704, 1024)

0: 704x1024 198 Maizes, 45.5ms
Speed: 4.0ms preprocess, 45.5ms inference, 2.0ms postpr

In [31]:
def calculate_mre(predicted_values, true_values):
    if len(predicted_values) != len(true_values):
        raise ValueError("Lists must be of equal length")

    total_mre = 0

    for predicted, true in zip(predicted_values, true_values):
        if true != 0:
            relative_error = abs((predicted - true) / true)
            total_mre += relative_error

    if len(true_values) > 0:
        mean_mre = total_mre / len(true_values)
        return mean_mre
    else:
        return 0  # Avoid division by zero if the lists are empty

In [32]:
def count_bounding_boxes(directory_path):
    total_bounding_boxes = []

    # Iterate through all text files in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory_path, filename)

            # Read the text file and count the lines
            with open(file_path, 'r') as file:
                bounding_box_count = sum(1 for line in file)

            total_bounding_boxes.append(bounding_box_count)

    return total_bounding_boxes

# Example usage:
directory_path = 'uav/test/labels/'
gt = count_bounding_boxes(directory_path)


In [33]:
def calculate_mae(predicted_values, true_values):
    # Ensure the input lists have the same length
    assert len(predicted_values) == len(true_values), "Input lists must have the same length"

    # Calculate absolute differences for each example
    absolute_differences = [abs(true - pred) for pred, true in zip(predicted_values, true_values)]

    # Calculate the mean absolute error
    mae = sum(absolute_differences) / len(absolute_differences)

    return mae

In [34]:
def calculate_rmse(predicted_values, true_values):
    # Ensure the input lists have the same length
    assert len(predicted_values) == len(true_values), "Input lists must have the same length"

    # Calculate squared differences for each example
    squared_differences = [(true - pred)**2 for pred, true in zip(predicted_values, true_values)]

    # Calculate the mean squared error
    mse = sum(squared_differences) / len(squared_differences)

    return mse ** 0.5

In [35]:
calculate_mae(predictions, gt)

30.35483870967742

In [36]:
calculate_rmse(predictions, gt)

55.812185049503306

In [37]:
calculate_mre(predictions, gt)

0.10270379561451563